# Generating stimuli for replication of Saffran et al. 1999

The final project for the Research Methods module (ECS719P) is to replicate an experiment from a paper. We decided to replicate the 1st experiment from Saffran et al. 1999 ("Statistical learning of tone sequences by human infants and adults"). This notebook generates the stimuli for the experiment.

## The stimuli

The paper uses eleven 0.33 second pure tones within an (almost) chromatic scale starting at middle C (A# was skipped). These are the "sylables". For each of 2 languages 6 "tone words" are created, each one from 3 syllables. 6 sentences are composed from 18 words each. These are concatenated to create 7 minute "tone story" for each language. Note that there is no mark for word / sentence boundary!

## Issues

- The paper uses 11 tones, but they are not 11 chromatic tones starting from middle C, the note A# was skipped.
- The way sentences are composed is unclear. We concluded that each word is presented 3 times in a sentence (with no consecutive pairs), but it is not obvious from the text.
- Should we add a short fade in and fade out for each note to eliminate the clicking sound? Currently there are no fades.
- There are two randomizations of the test trials. It is unclear if everything should be randomized (both order of trials and order of pairs) or only the order of the trials. Current the trials are fully randomized.

In [1]:
from collections import Iterable
from datetime import datetime
import itertools
import json
import functools
import os
import random

import soundfile as sf
import numpy as np

### General preperation for current run

In [2]:
OUTPUT_BASE_DIR = 'output'
LOG_FILE = 'log.txt'

run = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
print(f'Current run: {run}')

if not os.path.exists(OUTPUT_BASE_DIR):
    os.mkdir(OUTPUT_BASE_DIR)
output_dir = os.path.join(OUTPUT_BASE_DIR, run)
os.mkdir(output_dir)

python_print = print


def print(*args, **kwargs):
    """
    Both print and write everything to log file.
    """
    python_print(*args, **kwargs)
    # Open in 'append' mode
    with open(os.path.join(output_dir, LOG_FILE), 'a') as f:
        kwargs['file'] = f
        python_print(*args, **kwargs)

Current run: 2017_01_21__15_09_56


## Symbolic data generation

Most of the work will involve symbolic representation of the materials. The tones will be marked with numbers (0 to 10).

### Utility functions

In [3]:
# Functions to print notes, words, sentences, and stories

def note_text(note):
    texts = [
        'C',
        'C#',
        'D',
        'D#',
        'E',
        'F',
        'F#',
        'G',
        'G#',
        'A',
        'B',  # Notice that A# is skipped
    ]
    return texts[note]


def word_text(word):
    return ''.join(note_text(note) for note in word)


def sentence_text(sentence):
    return ' '.join(word_text(word) for word in sentence)


def story_text(story):
    return ' | '.join(sentence_text(sentence) for sentence in story)


# Other utilities

def contains_consecutive_pairs(iterable):
    for x, y in zip(iterable, iterable[1:]):
        if x == y:
            return True
    return False


assert not contains_consecutive_pairs([1, 2, 3, 4])
assert not contains_consecutive_pairs([])
assert contains_consecutive_pairs([1, 2, 2, 3, 4])


def flatten(iterable):
    for item in iterable:
        if isinstance(item, Iterable):
            yield from flatten(item)
        else:
            yield item
            
            
assert list(flatten([1, [2, [3, 4]]])) == [1, 2, 3, 4]
assert list(flatten([1, [2, 3], [4, 5]])) == [1, 2, 3, 4, 5]

### Generating a symbolic "tone story"

The `languages` list holds the entire data structure for words, sentences, and stories for each language.

In [4]:
languages = [
    {
        'num': 1,
        'words': [
            (9, 2, 10),
            (2, 5, 4),
            (7, 8, 9),
            (5, 0, 6),
            (3, 4, 2),
            (0, 1, 2),
        ],
    },
    {
        'num': 2,
        'words': [
            (9, 1, 4),
            (6, 8, 4),
            (7, 0, 3),
            (1, 10, 9),
            (1, 5, 2),
            (8, 10, 9),
        ],
    }
]

# Sanity check that the words are correct
# Compare the output to the paper
for language in languages:
    print(f"language {language['num']} words:", end=' ')
    print(sentence_text(language['words']))
print()

language 1 words: ADB DFE GG#A FCF# D#ED CC#D
language 2 words: AC#E F#G#E GCD# C#BA C#FD G#BA



In [5]:
def create_sentence(words):
    # Generate sentences until there are no consecutive words
    while True:
        sentence = words * 3
        random.shuffle(sentence)
        if not contains_consecutive_pairs(sentence):
            return sentence


for language in languages:
    # 6 sentences per language
    language['sentences'] = [create_sentence(language['words']) for _ in range(6)]

In [6]:
for language in languages:
    # 24 sentences in a story
    language['story'] = random.choices(language['sentences'], k=24)

### Generating symbolic trials (and practice trials)

Each participants hear 36 trials, each one contains a word the participant is familiar with and a word from the opposite language. The participant than mark (on paper) which word was more familiar. There are two randomized versions of the trials.

In [7]:
def generate_random_trials(languages):
    languages_words = [l['words'] for l in languages]
    trials = list(itertools.product(*languages_words))
    random.shuffle(trials)
    for trial in trials:
        trial = list(trial)
        random.shuffle(trial)
        yield trial
        
# Two randomized trials
trials = [list(generate_random_trials(languages)) for _ in range(2)]

"All subjects heard three practice trials prior to testing".

In [8]:
practice_trials = list(itertools.islice(generate_random_trials(languages), 3))

### Exporting the generated data

In [9]:
data = {
    'languages': languages,
    'trials': trials,
    'practice_trials': practice_trials,
}

with open(os.path.join(output_dir, 'data.json'), 'w') as f:
    json.dump(data, f)

### Calculating the transition probabilities matrix and from that languages and words stats

The generated data structures are not modified here. All of the code below only analyse the generated data and spits out some stats that were used in the paper.

In [10]:
def calculate_transitional_probabilities(story):
    """
    Calculate a 11 * 11 matrix of transitional probabilities in the "tone story".
    Cell [i, j] in the matrix is the probability of transitioning to note j after
    note i.
    """
    frequency_matrix = np.zeros((11, 11))
    flat_story = list(flatten(story))
    for x, y in zip(flat_story, flat_story[1:]):
        frequency_matrix[x, y] += 1
    # Normalized rows
    trans_probs = frequency_matrix / frequency_matrix.sum(axis=1)[:, np.newaxis]
    return np.nan_to_num(trans_probs)


def calculate_within_words_mask(words):
    """
    Calculate a 11 * 11 mask matrix (elements are either 0 or 1) that mark the
    within words transitions. AKA, if a transition from i to j is presented in a
    word then the matrix [i, j] value is 1, otherwise it's 0.
    """
    within_words_mask = np.zeros((11, 11))
    for word in words:
        within_words_mask[word[0], word[1]] = 1
        within_words_mask[word[1], word[2]] = 1       
    return within_words_mask


def word_transitional_probabilities(word, trans_prob):
    """
    Calculate the average transitional probabilities of a word.
    """
    return (trans_probs[word[0], word[1]] + trans_probs[word[1], word[2]]) / 2


for language in languages:
    trans_probs = calculate_transitional_probabilities(language['story'])
    within_words_mask = calculate_within_words_mask(language['words'])

    # Printing the results
    header = f"language {language['num']} probabilities"
    print(header)
    print('-' * len(header))

    for name, is_within in [('within', 1), ('between', 0)]:
        values = trans_probs[within_words_mask == is_within]
        print(f'{name} words: min {min(values):.3f}; max {max(values):.3f}; mean {np.mean(values):.3f}')
    
    print('words transitional probabilities (averaged over the two within word transitions)')
    for word in language['words']:
        print(f'- {word_text(word)}: {word_transitional_probabilities(word, trans_probs):.3f}')
    print()

language 1 probabilities
------------------------
within words: min 0.250; max 1.000; mean 0.652
between words: min 0.000; max 0.319; mean 0.029
words transitional probabilities (averaged over the two within word transitions)
- ADB: 0.476
- DFE: 0.434
- GG#A: 1.000
- FCF#: 0.500
- D#ED: 0.753
- CC#D: 0.750

language 2 probabilities
------------------------
within words: min 0.333; max 1.000; mean 0.679
between words: min 0.000; max 0.458; mean 0.032
words transitional probabilities (averaged over the two within word transitions)
- AC#E: 0.404
- F#G#E: 0.750
- GCD#: 1.000
- C#BA: 0.667
- C#FD: 0.667
- G#BA: 0.750



## Audio generation

### Audio utility functions

Note frequencies are copied from PureData `mtof` object.

In [11]:
SAMPLE_RATE = 44100
AMPLITUDE = 0.8  # Just be on the safe side...


def note_freq(note):
    freqs = [
        261.6,  # middle C
        277.1,
        293.6,
        311.1,
        329.6,
        349.2,
        369.9,
        391.9,
        415.3,
        440,  # A
        493.8,  # B, notice that A# was skipped
    ]
    return freqs[note]


# Sanity check that A is indeed 440
assert(note_text(9) == 'A')
assert(note_freq(9) == 440)


@functools.lru_cache(maxsize=11)
def note_audio(note):
    """
    Generating an array of 0.33 second of that note.
    """
    length_in_samples = int(SAMPLE_RATE * 0.33)
    freq = note_freq(note)
    x = np.arange(length_in_samples)
    fade_samples = int(0.01 * SAMPLE_RATE)
    envelope = np.concatenate([
        np.linspace(0, 1, fade_samples),
        np.ones(length_in_samples - 2 * fade_samples),
        np.linspace(1, 0, fade_samples),
    ])
    return AMPLITUDE * envelope * np.sin(2 * np.pi * freq * x / SAMPLE_RATE)


@functools.lru_cache(maxsize=2)
def silence(length_in_seconds):
    return np.zeros(int(length_in_seconds * SAMPLE_RATE))


def word_audio(word):
    return np.concatenate([note_audio(note) for note in word])


def trial_audio(trial):
    word_a, word_b = trial
    return np.concatenate([
        word_audio(word_a),
        silence(0.75),
        word_audio(word_b),
        silence(5),
    ])


def trials_audio(trials):
    return np.concatenate([trial_audio(trial) for trial in trials])


def export(filename, audio):
    filepath = os.path.join(output_dir, filename)
    sf.write(filepath, audio, SAMPLE_RATE)

### Generate stories

In [12]:
%%time

for language in languages:
    flat_story = flatten(language['story'])
    audio_story = np.concatenate([note_audio(note) for note in flat_story])
    export(f"language_{language['num']}_story.wav", audio_story)

CPU times: user 377 ms, sys: 203 ms, total: 580 ms
Wall time: 795 ms


### Generate trials

There are 0.75 seconds silence between the words and 5 seconds between trials.

In [13]:
%%time

export('trials_a.wav', trials_audio(trials[0]))
export('trials_b.wav', trials_audio(trials[1]))
export('practice_trials.wav', trials_audio(practice_trials))

CPU times: user 263 ms, sys: 287 ms, total: 550 ms
Wall time: 694 ms
